<a href="https://colab.research.google.com/github/namritha-maddali/Baymin/blob/main/Baymin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai tiktoken langchain faiss-cpu pypdf streamlit streamlit_chat
!npm install localtunnel

In [ ]:
%%writefile app.py

import streamlit as st

import openai
import os
os.environ['OPENAI_API_KEY'] = 'your-api-key'
openai_api_key = 'your-api-key'

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

st.set_page_config(page_title="Your Person First Aid Companion", page_icon="🏥")
st.title('Hello, I am Baymin!')
st.caption('I am like Baymax, your personal healthcare companion but mostly for first aid')
tab1= st.tabs(["First Aid"])

loader = PyPDFLoader('/content/first-aid.pdf')
pages = loader.load_and_split()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
documents = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(temperature=1, model='gpt-3.5-turbo')

db = FAISS.from_documents(documents,embeddings)

baymax_template = """ You are an extremely well trained paramedic who knows everything to do in different kinds of \
    emergency situations that require first aid. You have all the information regarding what first aid precautions and \
    measures that must be taken in different kinds of situations. You can use the document being given through the \
    document along with the information you already have regarding emergency situations and first aid. \
    You are the first point of contact for the person performing first aid on someone who needs it. \
    You give precise, accurate and short answers in steps format that to tell the steps regarding what must be done as \
    it is an emergency situation where the person must act fast.

    context: {context}?
    here is a prompt asked by a layman bystander who wants to perform first aid immediately:
    {question}
"""

memory = ConversationBufferMemory(
    llm = llm,
    output_key = 'answer',
    memory_key = 'chat_history',
    return_messages = True)

retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k": 5, "include_metadata": True})


messages = [SystemMessagePromptTemplate.from_template(baymax_template)]
qa_prompt = ChatPromptTemplate.from_messages(messages)

chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    memory = memory,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
    get_chat_history = lambda h : h,
    combine_docs_chain_kwargs = {'prompt': qa_prompt},
    verbose = False,
)

if "chat_history" not in st.session_state:
    st.session_state["chat_history"] = []
if "generated" not in st.session_state:
    st.session_state["generated"] = ["Hello, I am Baymin. Your personal first aid companion. I was alerted to the need for medical attention when you said, \"ow\""]
if "past" not in st.session_state:
    st.session_state["past"] = ["Ow"]

response_container = st.container()      # container for the chat history
container = st.container()               # container for the user's text input

def generate_response(query):
    result = chain({"question": query, "chat_history": st.session_state["chat_history"]})
    # st.write(result['answer'])
    st.session_state['chat_history'] = [(query, result['answer'])]

    # with st.sidebar:
        # st.write(result['source_documents'])

    return result['answer']

with container:
    user_input = st.chat_input(placeholder = " what emergency are you facing?")
    if user_input:
      output = generate_response(user_input)
      st.session_state["past"].append(user_input)
      st.session_state["generated"].append(output)
      st.session_state["chat_history"] = [(user_input, output)]

with response_container:
    for i in range(len(st.session_state["generated"])):
        st.chat_message('user_input', avatar='🧑🏻‍⚕️').write(st.session_state["past"][i],key=str(i) + '_user')
        st.chat_message('user', avatar='🤖').write(st.session_state["generated"][i],key=str(i))

In [9]:
!curl https://ipv4.icanhazip.com/
!streamlit run /content/app.py &>/content/logs.txt &

34.68.49.165


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.643s
your url is: https://neat-rats-trade.loca.lt
